# Compare output snow depth estimates to SNOTEL observations

In [ ]:
import os, glob
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray as rxr
import pandas as pd
import geopandas as gpd
from shapely import wkt
import re

In [ ]:
site_name = 'MCS'
date = '20240420'
sd_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/snow_depth_maps'
out_fn = os.path.join(sd_dir, f'{site_name}_{date}_snow_depth_estimates_comparison.png')

sd_fns = sorted(glob.glob(os.path.join(sd_dir, '*differences.tif')))
print('Snow depth map file names:')
print(sd_fns)

snotel_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}/snotel/'
snotel_fn = glob.glob(os.path.join(snotel_dir, '*_adj.csv'))[0]
print('\nSNOTEL file name:')
print(snotel_fn)

snotel_info_fn = glob.glob(os.path.join(snotel_dir, '*_site_info.csv'))[0]
print('\nSNOTEL site info file name:')
print(snotel_info_fn)

In [ ]:
# Load SNOTEL site info
snotel_info = pd.read_csv(snotel_info_fn)
snotel_info['geometry'] = snotel_info['geometry'].apply(wkt.loads)
snotel_info_gdf = gpd.GeoDataFrame(snotel_info, geometry='geometry', crs='EPSG:4326')
# Load SNOTEL data
snotel = pd.read_csv(snotel_fn)
snotel['datetime'] = pd.to_datetime(snotel['datetime'])
# Identify snow depth for date
dt = pd.to_datetime(f'{date[0:4]}-{date[4:6]}-{date[6:8]} 00:00:00+00:00')
snotel['dt'] = snotel['datetime'] - dt
snotel_sd = snotel.loc[np.abs(snotel['dt']) == np.abs(snotel['dt']).min(), 'SNWD_m'].values[0]
print(f'SNOTEL snow depth = {snotel_sd} m')

In [ ]:
# Set up figure
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})
fig, ax = plt.subplots(len(sd_fns), 1, figsize=(10, 4*len(sd_fns)))

# Iterate over snow depth maps
for i, sd_fn in enumerate(sd_fns):
    # Load snow depth map
    sd = rxr.open_rasterio(sd_fn)
    sd_crs = f'EPSG:{sd.rio.crs.to_epsg()}'
    sd = xr.where(sd==sd.attrs['_FillValue'], np.nan, sd)
    sd_data = np.ravel(sd.data)

    # sample snow depth map at snotel point
    snotel_info_gdf_reproj = snotel_info_gdf.to_crs(sd_crs)
    snotel_reproj = snotel_info_gdf.to_crs(sd_crs)
    sample_x = snotel_reproj['geometry'].values[0].coords.xy[0][0]
    sample_y = snotel_reproj['geometry'].values[0].coords.xy[1][0]
    sample_sd = sd.sel(x=sample_x, y=sample_y, method='nearest').data[0]
    sample_snotel_diff = sample_sd - snotel_sd
    
    # plot
    ax[i].hist(sd_data, color='grey', bins=50)
    ymin, ymax = ax[i].get_ylim()
    ax[i].plot([snotel_sd, snotel_sd], [ymin, ymax], '-c', linewidth=2, label='SNOTEL')
    ax[i].plot([sample_sd, sample_sd], [ymin, ymax], '-k', linewidth=1, label='dDEM')
    ax[i].legend(loc='upper right')
    ss_method = ('roads' if 'ss-roads' in os.path.basename(sd_fn) else 'Sentinel-2 NDSI')
    methods = os.path.basename(sd_fn).split('_')
    coreg_method = [x for x in methods if 'Nuth' in x][0]
    ss_method = [x for x in methods if ('ss-roads' in x) or ('ss-S2' in x)][0]
    mask_method = [x for x in methods if 'masked-' in x][0]
    nmad_thresh = mask_method.split('NMAD')[1].split('m')[0]
    refl_thresh = mask_method.split('refl')[1]
    title = (f'Stable surfaces = {ss_method}'
             f'\nCoregistration method: {coreg_method}'
             f'\nMasking method: NMAD <= {nmad_thresh} m, Refl >= {refl_thresh}'
             f'\ndDEM - SNOTEL = {np.round(sample_snotel_diff, 2)} m')
    ax[i].set_xlim(-10, 10)
    ax[i].set_ylim(ymin, ymax)
    ax[i].set_title(title)
ax[i].set_ylabel('Count')

ax[-1].set_xlabel('Snow depth estimate [m]')

fig.tight_layout()
plt.show()

# Save figure
fig.savefig(out_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', out_fn)